### t5-small-korean Finetuning 실습
- 인코더-디코더 구조 특성상 문장 요약 태스크에 적합한 T5 모델로 미세 조정하여 한국어 기사 본문을 요약하는 모델 생성
- Causal LM인 Mistral 모델로 생성한 요약 모델과 성능 비교해보기
- Base model: "paust/pko-t5-small"
- 데이터셋 출처: "daekeun-ml/naver-news-summarization-ko"

In [ ]:
import torch

In [ ]:
# 데이터셋 로드 및 샘플링
!pip install datasets -qqq
from datasets import load_dataset
train_dataset = load_dataset("daekeun-ml/naver-news-summarization-ko", split='train').shuffle(seed=42).select(range(2000))
eval_dataset = load_dataset("daekeun-ml/naver-news-summarization-ko", split='validation').shuffle(seed=42).select(range(200))
test_dataset = load_dataset("daekeun-ml/naver-news-summarization-ko", split='test').shuffle(seed=42).select(range(200))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.2 MB/s eta 0:00:00


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/787 [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/66.3M [00:00<?, ?B/s]

validation.csv:   0%|          | 0.00/7.45M [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/8.17M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/22194 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2466 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2740 [00:00<?, ? examples/s]

In [ ]:
# 불필요한 컬럼 제거
train_dataset = train_dataset.remove_columns(['date', 'category', 'press', 'title', 'link'])
eval_dataset = eval_dataset.remove_columns(['date', 'category', 'press', 'title', 'link'])
test_dataset = test_dataset.remove_columns(['date', 'category', 'press', 'title', 'link'])

In [ ]:
print(train_dataset, eval_dataset, test_dataset)

Dataset({
    features: ['document', 'summary'],
    num_rows: 2000
}) Dataset({
    features: ['document', 'summary'],
    num_rows: 200
}) Dataset({
    features: ['document', 'summary'],
    num_rows: 200
})


In [ ]:
# 데이터 전처리 함수
def preprocess_function(examples):
    # inputs 텍스트 앞에 "summarize: "을 붙임 (T5 학습 및 추론 형식)
    inputs = ["summarize: " + doc for doc in examples["document"]]
    # 정답은 요약문 가져옴
    targets = examples["summary"]

    # 먼저 모델 학습용 inputs에 위에서 가져온 "summarize: " + 본문을 토큰화해서 저장
    model_inputs = tokenizer(inputs, max_length=512, padding="max_length", truncation=True)

    # 요약문 토큰화해서 라벨에 저장
    labels = tokenizer(targets, max_length=128, padding="max_length", truncation=True)

    # 라벨의 토큰 id 중 패딩 토큰을 -100으로 변경해야 함 (그래야 Loss 계산에서 무시됨)  # 생성된 정답 토큰 중 패딩 토큰을 -> -100 으로 변경
    labels["input_ids"] = [ [-100 if token == tokenizer.pad_token_id else token for token in label] for label in labels["input_ids"] ]

    # 라벨의 토큰 id를 모델 inputs의 'label'로 지정
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
# 토크나이저 설정
from transformers import AutoTokenizer

model_id = "paust/pko-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_id)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/1.95k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.92M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

In [ ]:
# train, val, test 데이터셋 전처리 수행
train_dataset = train_dataset.map(preprocess_function, batched=True)
eval_dataset = eval_dataset.map(preprocess_function, batched=True)
test_dataset = test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:
# 허깅페이스 로그인
from huggingface_hub import login

# 허깅페이스 허브 로그인
token = "****"  # 허깅페이스 액세스 토큰 입력
login(token=token)

In [ ]:
!pip install evaluate -qqq
!pip install rouge_score -qqq
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model, AutoPeftModelForSeq2SeqLM
import evaluate
from tqdm import tqdm
import numpy as np
import rouge_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
# 모델 불러오기
model = AutoModelForSeq2SeqLM.from_pretrained(model_id, device_map="auto")
# model = prepare_model_for_kbit_training(model)

config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/383M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/383M [00:00<?, ?B/s]

In [ ]:
# 로라 config 설정 및 모델에 결합
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q", "v", "k", "o", "wi", "wo"],
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_2_SEQ_LM"
)

model = get_peft_model(model, lora_config)

In [ ]:
# wandb 설정
import wandb
wandb.login(key='77a08abc9cfd76e4b78603826bd7a863487240ac')
run = wandb.init(project='t5-small-korean-news-summarizer', job_type='training', anonymous='allow')

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: dgriii0606 (dg-test) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [ ]:
# 학습 어규먼트 설정
training_args = Seq2SeqTrainingArguments(
    output_dir="korean-news-summarizer",
    learning_rate=3e-4,
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    save_strategy="epoch",
    evaluation_strategy="epoch",
    logging_dir="logs",
    logging_strategy="steps",
    logging_steps=10,
    report_to="wandb",
    predict_with_generate=True
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# 평가 메트릭 함수 생성
rouge = evaluate.load('rouge')

def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    # 각 배열을 Python 리스트로 변환
    predictions = predictions.tolist()
    labels = labels.tolist()

    # 위에서 패딩 토큰 -> -100 처리한 부분을 다시 원래 패딩 토큰으로 변환  # labels중 -100을 패딩 토큰(pad_token_id)으로 변환
    labels = [ [tokenizer.pad_token_id if token == -100 else token for token in label] for label in labels ]  # 2차원 배열에 대한 리스트 컴프리헨션

    # batch_decode로 한 번에 디코딩  # 배치 단위로 디코딩
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # ROUGE 점수 계산
    rouge_scores = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_aggregator=True)

    return rouge_scores

In [ ]:
# 트레이너 설정
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer),
    compute_metrics=compute_metrics

)

<ipython-input-57-e7a1550888f5>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,0.695600,0.720194,0.305070,0.082279,0.296962,0.296996


TrainOutput(global_step=250, training_loss=0.6835206832885742, metrics={'train_runtime': 51.1865, 'train_samples_per_second': 39.073, 'train_steps_per_second': 4.884, 'total_flos': 439999266816000.0, 'train_loss': 0.6835206832885742, 'epoch': 1.0})

In [ ]:
# 테스트 데이터셋 평가
trainer.evaluate(test_dataset)

{'eval_loss': 0.6840291023254395,
 'eval_model_preparation_time': 0.0142,
 'eval_rouge1': 0.2581140851107954,
 'eval_rouge2': 0.06744222444222445,
 'eval_rougeL': 0.25196706254272033,
 'eval_rougeLsum': 0.25144132804330155,
 'eval_runtime': 13.0397,
 'eval_samples_per_second': 15.338,
 'eval_steps_per_second': 1.917}

In [ ]:
sample_text = """

정월대보름(2월12일)을 앞두고 서민들의 시름이 깊어지고 있다.
고물가 장기화에 장바구니 가격 부담이 갈수록 커지는 가운데 정월대보름에 챙겨먹는 오곡밥과 부럼 등 재료 가격마저 크게 올랐기 때문이다. 대형마트는 국산 재료 값이 급등하자 일부 품목을 수입산으로 대체하고 있다.
10일 대형마트 업계에 따르면 오곡밥 주재료인 붉은팥, 찹쌀, 서리태, 수수, 차조 등 국산 잡곡 시세가 일제히 상승했다.
특히 잡곡밥에 들어가는 붉은팥 가격이 전년 대비 50%가량 뛰었고 찹쌀도 23% 이상 급등했다. 부럼 재료인 은행과 땅콩 가격 역시 17%가량 올랐다. 국산 건나물도 상황은 마찬가지다.
호박과 고구마순의 가격이 각각 20%, 10% 이상 뛰었고 기획상품으로 내놓는 건나물 4종 세트 역시 전년 대비 평균 5~10% 올랐다.
유통업계에서는 정월대보름 주요 품목 가격이 오른 이유로 재배 면적 축소에 따른 생산량 감소, 폭염 등 이상기후로 인한 작황 부진, 고물가 장기화에 집밥 수요 급증 등의 영향 등을 꼽고 있다.
이에 따라 대형마트들은 고객들의 장바구니 물가 부담을 덜어주기 위해 일부 품목을 수입산으로 대체하고 있다.
붉은팥과 호두, 땅콩 등이 대표적이다.
롯데마트는 오는 12일까지 캐나다·페루산 붉은팥을 1㎏·1.2㎏들이 1봉당 각각 7990원에 선보인다.
또 중국산 볶음 피땅콩(450g)과 미국산 피호두(미국산·300g)를 2개 이상 구매하면 개당 2000원 할인한 각각 5990원에 판다.
홈플러스는 같은 기간 캐나다산 붉은팥을 1봉(600ｇ)당 9990원에 팔고 1봉을 사면 1봉을 덤으로 주는 ‘1+1’ 행사를 연다.
또 미국산 호두(300g)와 중국산 볶음피땅콩(500g)을 4990원에 내놓고, 미국산 피스타치오(100g)를 포함, 대보름 부럼세트(총 280ｇ)를 6990원에 판매한다.
이마트 역시 미국산 피호두(300ｇ)와 중국산 볶음 피땅콩(480ｇ)을 25% 할인한 5235원에 내놓는다.
한편 한국물가정보가 발표한 정월 대보름 주요 10개 품목 가격을 보면 전통시장의 합산 가격은 지난해 대비 6.2% 오른 13만9700원, 대형마트는 8.0% 오른 18만5220원으로 나타났다.
가격 상승 폭이 가장 큰 품목은 오곡밥 재료 중 붉은팥으로 1되(800g)가 전통시장에서는 전년 대비 45.5% 오른 1만6000원, 대형마트에서는 45.0% 오른 2만1920원이었다.
찹쌀은 1되(800g) 가격이 전통시장 기준 3200원으로 지난해보다 23.1% 올랐고, 대형마트에서는 5040원으로 28.6% 뛰었다.
검정콩 1되(720g)는 지난해보다 전통시장·대형마트 판매 가격이 각각 7.1%, 5.2% 올랐다.
부럼 재료 중에서는 은행과 땅콩 가격이 크게 올랐다. 은행 1되(600g) 가격은 전통시장 7000원, 대형마트 9840원으로 지난해보다 각각 16.7%, 15.2% 뛰었다.
땅콩 1되(400g)는 전통시장 가격이 1만원으로 지난해보다 11.1% 올랐고, 대형마트는 1만3560원으로 13.4% 인상됐다.

"""

In [ ]:
# t5 인풋 형식으로 인풋 텍스트 저장해서 토크나이저로 토큰화
input_text = "summarize: " + sample_text
inputs = tokenizer(input_text, return_tensors="pt", max_length=1024, truncation=True).to("cuda")

# 아웃풋 생성하고 아웃풋 다시 디코딩
outputs = model.generate(**inputs, max_length=128, num_beams=3, temperature=0.7)
summary = tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
summary

'10일 대형마트 업계에 따르면 오곡밥 주재료인 붉은팥, 찹쌀, 서리태, 수수, 차조 등 국산 잡곡 시세가 일제히 상승하며 오곡밥과 부럼 등 재료 가격마저 크게 올랐기 때문에 일부 품목을 수입산으로 대체하고 있다.'

In [ ]:
# 베이스 모델과 병합
merged_model = model.merge_and_unload()
merged_model.save_pretrained("merged-summarizer")
tokenizer.save_pretrained("merged-summarizer")

('merged-summarizer/tokenizer_config.json',
 'merged-summarizer/special_tokens_map.json',
 'merged-summarizer/tokenizer.json')

In [ ]:
from huggingface_hub import HfApi

api = HfApi()

repo_id = "edgeun/t5-small-korean-news-summarizer"
api.create_repo(repo_id, private=False)

merged_model.push_to_hub(repo_id)
tokenizer.push_to_hub(repo_id)

model.safetensors:   0%|          | 0.00/383M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/edgeun/t5-small-korean-news-summarizer/commit/d1ee52e8d35453e0f0a8364354500f29bcbbdaba', commit_message='Upload tokenizer', commit_description='', oid='d1ee52e8d35453e0f0a8364354500f29bcbbdaba', pr_url=None, repo_url=RepoUrl('https://huggingface.co/edgeun/t5-small-korean-news-summarizer', endpoint='https://huggingface.co', repo_type='model', repo_id='edgeun/t5-small-korean-news-summarizer'), pr_revision=None, pr_num=None)